# Serialize/Deserialize

The [EODataAccessGateway](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway) class provides methods to save a [SearchResult](../../api_reference/searchresult.rst#eodag.api.search_result.SearchResult) object to a GeoJSON file or to load a GeoJSON file to a [SearchResult](../../api_reference/searchresult.rst#eodag.api.search_result.SearchResult). These methods come in handy to save the state of a search and restore it later.

You will learn how to:
- [Serialize](#Serialize)  
- [Deserialize](#Deserialize)

The credentials are required in this notebook to download a product from *PEPS*.

In [1]:
import os

os.environ["EODAG__PEPS__AUTH__CREDENTIALS__USERNAME"] = "PLEASE_CHANGE_ME"
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__PASSWORD"] = "PLEASE_CHANGE_ME"

In [2]:
from eodag import EODataAccessGateway
from eodag import setup_logging

setup_logging(2)

dag = EODataAccessGateway()
dag.set_preferred_provider("peps")

2024-07-31 11:56:42,553 eodag.config                     [INFO    ] Loading user configuration from: /home/anesson/.config/eodag/eodag.yml
2024-07-31 11:56:42,649 eodag.core                       [INFO    ] Locations configuration loaded from /home/anesson/.config/eodag/locations.yml


## Serialize

The method [serialize()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.serialize) allows to save a [SearchResult](../../api_reference/searchresult.rst#eodag.api.search_result.SearchResult) as a *GeoJSON* file.

In [3]:
search_results = dag.search(
    collection="S2_MSI_L1C",
    start="2021-03-01",
    end="2021-03-31",
    geom={"lonmin": 1, "latmin": 43, "lonmax": 2, "latmax": 44},
    items_per_page=2
)

2024-07-31 11:56:43,213 eodag.core                       [INFO    ] Searching collection 'S2_MSI_L1C' on provider: peps
2024-07-31 11:56:43,218 eodag.search.qssearch            [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2021-03-01&completionDate=2021-03-31&geometry=POLYGON ((1.0000 43.0000, 1.0000 44.0000, 2.0000 44.0000, 2.0000 43.0000, 1.0000 43.0000))&productType=S2MSI1C&maxRecords=2&page=1


In [4]:
search_results

SearchResult([EOProduct(id=S2B_MSIL1C_20210328T103629_N0500_R008_T31TDH_20230602T033834, provider=peps),
              EOProduct(id=S2B_MSIL1C_20210328T103629_N0500_R008_T31TCJ_20230602T033834, provider=peps)])

A folder is created to save the output of this notebook.

In [5]:
workspace = "eodag_workspace_serialize_deserialize"
if not os.path.isdir(workspace):
    os.mkdir(workspace)

In [6]:
output_file = os.path.join(workspace, "search_results.geojson")

dag.serialize(
    search_results, 
    filename=output_file
)

'eodag_workspace_serialize_deserialize/search_results.geojson'

## Deserialize

There are two methods offered by [EODataAccessGateway](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway) to load a search result saved as a GeoJSON:

* [deserialize()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.deserialize): it  simply recreates a [SearchResult](../../api_reference/searchresult.rst#eodag.api.search_result.SearchResult) and the [EOProduct](../../api_reference/eoproduct.rst#eodag.api.product._product.EOProduct) it contained
* [deserialize_and_register()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.deserialize_and_register): it also recreates a [SearchResult](../../api_reference/searchresult.rst#eodag.api.search_result.SearchResult) but additionally registers for each [EOProduct](../../api_reference/eoproduct.rst#eodag.api.product._product.EOProduct) the information it requires to download itself

In [7]:
deserialized_search_results = dag.deserialize(output_file)
deserialized_search_results

SearchResult([EOProduct(id=S2B_MSIL1C_20210328T103629_N0500_R008_T31TDH_20230602T033834, provider=peps),
              EOProduct(id=S2B_MSIL1C_20210328T103629_N0500_R008_T31TCJ_20230602T033834, provider=peps)])

In [8]:
deserialized_search_results[0].download()

RuntimeError: EO product is unable to download itself due to lacking of a download plugin

Trying to download a product from a [SearchResult](../../api_reference/searchresult.rst#eodag.api.search_result.SearchResult) that was loaded with [serialize()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.deserialize) since it doesn't try to configure each product so that it can be downloaded.

In [9]:
deserialized_and_registered = dag.deserialize_and_register(output_file)
deserialized_and_registered

SearchResult([EOProduct(id=S2B_MSIL1C_20210328T103629_N0500_R008_T31TDH_20230602T033834, provider=peps),
              EOProduct(id=S2B_MSIL1C_20210328T103629_N0500_R008_T31TCJ_20230602T033834, provider=peps)])

In [10]:
product_path = deserialized_and_registered[0].download(
    output_dir=workspace,
)

0.00B [00:00, ?B/s]

2024-07-31 11:57:11,106 eodag.download.base              [INFO    ] Download url: https://peps.cnes.fr/resto/collections/S2ST/2d85d4c8-34c1-55dc-b43c-0ab0fe4c2b97/download
2024-07-31 12:00:13,652 eodag.download.base              [INFO    ] Extraction activated
2024-07-31 12:00:23,734 eodag.download.base              [INFO    ] Deleting archive S2B_MSIL1C_20210328T103629_N0500_R008_T31TDH_20230602T033834.zip
2024-07-31 12:00:23,822 eodag.product                    [INFO    ] Remote location of the product is still available through its 'remote_location' property: https://peps.cnes.fr/resto/collections/S2ST/2d85d4c8-34c1-55dc-b43c-0ab0fe4c2b97/download


Downloading the product with [deserialize_and_register()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.deserialize_and_register) works as expected.